This notebook to test the results of a trained segmentation model. This notebook will apply the segmentation model to any GLIMS_ID and produce a list of segmentations.

In [ ]:
import sys
from tqdm import tqdm
import os
from helpers import read, preprocess
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import rasterio
from rasterio.mask import mask

In [ ]:
from skimage.filters import gaussian
from datetime import datetime
import cv2
import imageio.v2 as imageio

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from inference.cnn import UNet, conv_block
import pandas as pd
import torchvision
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [ ]:
torch_model = torch.load("model", map_location=torch.device('cpu')) #for cpu
#torch_model = torch.load("model") #for gpu
torch_model.to(device)
torch_model.eval()
pass

In [ ]:
#define inputs

common_bands = ['blue','green','red','nir','swir','thermal']
dim = (128,128)

data_label = "full_time_series_c02_t1_l2"
glacier_view_dir = os.path.join(os.path.expanduser("~"),
    "Desktop", "projects", "GlacierView")
ee_data_dir = os.path.join(glacier_view_dir,  "src", "earth_engine","data","ee_landing_zone",data_label)                      
landsat_dir = os.path.join(ee_data_dir, "landsat")
dem_dir = os.path.join(ee_data_dir, "dems")
masks_dir = os.path.join(glacier_view_dir, "src","segmentation","training","data","masks_staging_2")


In [ ]:
metadata_dir = os.path.join(glacier_view_dir,"src","earth_engine","data","processed_metadata",data_label)
filtered_inference_df = pd.read_csv(os.path.join(metadata_dir,"filtered_inference_data_summer_months.csv"))
print(filtered_inference_df.shape)
filtered_inference_df = filtered_inference_df[filtered_inference_df['geog_area_rollup'] == "Europe"]
hq_file_names = sorted(filtered_inference_df.file_name)
print(filtered_inference_df.shape)

In [ ]:
example_glac_idx = 130
example_file_idx = 2

example_glacier_name = os.path.join(landsat_dir,os.listdir(landsat_dir)[example_glac_idx])
example_glacier_dir = os.path.join(landsat_dir,example_glacier_name)

example_file_name = os.listdir(example_glacier_dir)[example_file_idx]


print(example_file_name)

img = rasterio.open(os.path.join(landsat_dir, example_glacier_name, example_file_name)).read()

fig, axs = plt.subplots(2,10,figsize=(20, 5))
for i in range(2):
    for j in range(10):
        if i==1 and j==9:
            pass
        else:
            axs[i,j].imshow(img[10*i+j])
plt.show()

In [ ]:
example_glac_idx = 129
example_file_idx = 28

example_glacier_name = os.path.join(landsat_dir,os.listdir(landsat_dir)[example_glac_idx])
example_glacier_dir = os.path.join(landsat_dir,example_glacier_name)

example_file_name = os.listdir(example_glacier_dir)[example_file_idx]


print(example_file_name)

img = rasterio.open(os.path.join(landsat_dir, example_glacier_name, example_file_name)).read()

fig, axs = plt.subplots(2,10,figsize=(20, 5))
for i in range(2):
    for j in range(10):
        if i==1 and j==9:
            pass
        else:
            axs[i,j].imshow(img[10*i+j])
plt.show()

In [ ]:
example_glac_idx = 129
example_file_idx = 0

example_glacier_name = os.path.join(landsat_dir,os.listdir(landsat_dir)[example_glac_idx])
example_glacier_dir = os.path.join(landsat_dir,example_glacier_name)

example_file_name = os.listdir(example_glacier_dir)[example_file_idx]


print(example_file_name)

img = rasterio.open(os.path.join(landsat_dir, example_glacier_name, example_file_name)).read()

fig, axs = plt.subplots(2,10,figsize=(20, 5))
for i in range(2):
    for j in range(10):
        if i==1 and j==9:
            pass
        else:
            axs[i,j].imshow(img[10*i+j])
plt.show()

In [ ]:
filtered_inference_df

In [ ]:

for glims_id in np.unique(filtered_inference_df.glims_id):

    PROB_THRESH = 0.5
    
    
    glacier_dir = os.path.join(landsat_dir, glims_id)
    dem_path = os.path.join(dem_dir, f"{glims_id}_NASADEM.tif")


    mask_file_name = f"{glims_id}.tif"
    mask = Image.open(os.path.join(masks_dir, mask_file_name))
    mask = np.expand_dims(np.array(mask),2)
    mask = {mask_file_name: mask}

    #read and preprocess images
    images = read.get_rasters(glacier_dir)
    single_key = list(images.keys())[0]
    original_sizes = [images[key].shape[:-1] for key in images if key in hq_file_names]
    
    nimages = preprocess.get_common_bands(images,common_bands)
    nimages = preprocess.normalize_rasters(nimages)
    nimages = preprocess.resize_rasters(nimages,dim)


    #read and preprocess dems
    dem = read.get_dem(dem_path)
    
    dem_key = list(dem.keys())[0]
    dem = preprocess.resize_rasters(dem, dim)
    dem = preprocess.normalize_rasters(dem)

    #preprocess masks
    mask = preprocess.resize_rasters(mask,dim)
    mask = list(mask.values())[0]

    # combine images and dems - high quality only
    hq_list = list(pd.Series(nimages.keys())[list(pd.Series(list(nimages.keys())).isin(hq_file_names))])
    combined_images_and_dems = [np.concatenate((nimages[file_name], dem[dem_key]),axis = 2) for file_name in sorted(hq_list)]

    #process data
    X = np.stack(combined_images_and_dems)
    X = np.nan_to_num(X, copy=True, nan=0.0)
    
    X_smoothed = X #gaussian(X, sigma = [0.5,0,0,0], mode = 'nearest') 
    image_file_names_ordered = sorted(hq_list)
    image_dates = [datetime.strptime(f.split("_")[1],'%Y-%m-%d') for f in image_file_names_ordered]

    inputs = torch.tensor(X_smoothed)

    inputs = inputs.permute(0,3,1,2)
    SMOOTH_FACTOR = 0
    
    green = inputs[:,1,:,:]
    swir = inputs[:,4,:,:]
    nir = inputs[:,3,:,:]
    
    ndsi = (green - swir)/(green + swir+SMOOTH_FACTOR)
    ndsi = ndsi.unsqueeze(dim=1)
    inputs = torch.cat((ndsi, inputs), dim=1)
    
    ndwi = (green - nir)/(green + nir+SMOOTH_FACTOR)
    ndwi = ndwi.unsqueeze(dim=1)
    inputs = torch.cat((ndwi, inputs), dim=1)
    prediction_dataset = TensorDataset(inputs) # create your datset
    prediction_dataloader = DataLoader(prediction_dataset, batch_size=64,shuffle=False) # create your dataloader

    predictions = []
    for i in tqdm(prediction_dataloader):
        with torch.no_grad():
            outputs = torch_model.forward(i[0].to(device=device,dtype=torch.float))
            outputs = torch.softmax(outputs, dim=1)
            outputs = outputs[:,1,:,:].unsqueeze(1)
        m = torch.nn.Threshold(PROB_THRESH, 0)
        predictions.append(m(outputs))
        
    predictions = torch.cat(predictions, dim=0)

    resized_predictions = []
    area_per_pred = []
    resized_imgs = []
    for index, size in enumerate(original_sizes):
        resize = torchvision.transforms.Resize(size, antialias=True)
        pred = resize(predictions[index])
        resized_imgs.append(pred.detach().cpu().numpy()[0])
        area = pred.sum().detach().cpu().numpy()
        area = area*0.0009 #to convert area
        area_per_pred.append(area)
        resized_predictions.append(resize(predictions[index]))


    df = pd.DataFrame(area_per_pred, image_dates)
    
    df[glims_id] = df[0]
    df.drop([0],axis=1)
    df =  df[glims_id]
    df = df.to_frame()
    df.index.names = ['Dates']
    df.to_csv(f"{glims_id}_areas.csv")

    ts_output_dir = "time_series"
    is_exist = os.path.exists(ts_output_dir)
    if not is_exist:
       os.makedirs(ts_output_dir)
    df.plot(figsize=(10,5), ylabel="Area in km sq.", xlabel="Years", title=f"Area vs Time plot for {glims_id} Glacier")
    mo = df.groupby(pd.PeriodIndex(df.index, freq="Y"))[df.columns[0]].mean()
    mo.plot()
    plt.legend(["Area at each time point","Annual Mean Area"])
    plt.savefig(os.path.join(ts_output_dir, f"{glims_id}.png"))

    #create GIF
    predictions = predictions.detach().cpu().numpy()

    gif_creation_dir = os.path.join("tmp", "gif_creation")
    is_exist = os.path.exists(gif_creation_dir)
    if not is_exist:
       os.makedirs(gif_creation_dir)

    gif_output_dir = "gifs"
    is_exist = os.path.exists(gif_output_dir)
    if not is_exist:
       os.makedirs(gif_output_dir)
    
    # gif_creation_dir = os.path.join(os.path.expanduser("~"), "PycharmProjects","glacier-view-analysis", "src", "segmentation","tmp","gif_creation")
    # gif_output_dir = os.path.join(os.path.expanduser("~"), "PycharmProjects","glacier-view-analysis", "src", "segmentation", "gifs")
    
    for f in os.listdir(gif_creation_dir):
        os.remove(os.path.join(gif_creation_dir, f))
    
    for i in range(X_smoothed.shape[0]):
        if i%1 == 0: #ignores 80% of images to run faster
            fig, axs = plt.subplots(2,3, figsize=(10,10)) ##update the number of suplots to equal the number of layers you want to display
            fig.suptitle(image_file_names_ordered[i])
            # axs[0].imshow(predictions[i,0,:,:],alpha=0.1)
            axs[0,0].imshow((X_smoothed[i,:,:,:][:,:,[2,1,0]]))
            axs[0,0].title.set_text('RGB')
            axs[0,1].imshow((ndsi[i,0,:,:]))
            axs[0,1].title.set_text('NDSI')
            axs[0,2].imshow(X_smoothed[i,:,:,:][:,:,5])
            axs[0,2].title.set_text('Thermal')
            axs[1,0].imshow(X_smoothed[i,:,:,:][:,:,6])
            axs[1,0].title.set_text('DEM')
            axs[1,1].imshow((X_smoothed[i,:,:,:][:,:,[2,1,0]]))
            axs[1,1].imshow(predictions[i,0,:,:],alpha=0.5)
            axs[1,1].title.set_text('Prediction')
            axs[1,2].imshow(mask)
            axs[1,2].title.set_text('Ground truth')
                        
    
            plt.savefig(os.path.join(gif_creation_dir,f'{image_file_names_ordered[i]}_final.png'), dpi = 100)
            # plt.show()
    
    with imageio.get_writer(os.path.join(gif_output_dir,f"{glims_id}_final_NEWEST.gif"), mode='I') as writer:
        for filename in sorted(os.listdir(gif_creation_dir)):
            image = imageio.imread(os.path.join(gif_creation_dir,filename))
            writer.append_data(image)


In [ ]:
for glims_id in np.unique(filtered_inference_df.glims_id[0]):

    PROB_THRESH = 0.5
    
    
    glacier_dir = os.path.join(landsat_dir, glims_id)
    dem_path = os.path.join(dem_dir, f"{glims_id}_NASADEM.tif")


    mask_file_name = f"{glims_id}.tif"
    mask = Image.open(os.path.join(masks_dir, mask_file_name))
    mask = np.expand_dims(np.array(mask),2)
    mask = {mask_file_name: mask}

    #read and preprocess images
    images = read.get_rasters(glacier_dir)
    single_key = list(images.keys())[0]
    original_sizes = [images[key].shape[:-1] for key in images if key in hq_file_names]
    
    nimages = preprocess.get_common_bands(images,common_bands)
    nimages = preprocess.normalize_rasters(nimages)
    nimages = preprocess.resize_rasters(nimages,dim)

    hq_list = list(pd.Series(nimages.keys())[list(pd.Series(list(nimages.keys())).isin(hq_file_names))])




In [ ]:
    import random
    fig, ax = plt.subplots(1,2, figsize = (10,6))
    key = random.choice(list(nimages.keys()))
    key_2 = random.choice(hq_list)
    print(key)
    ax[0].imshow(nimages[key][:,:,[2,1,0]])
    ax[0].set_title("Non filtered")
    ax[1].imshow(nimages[key_2][:,:,[2,1,0]])
    ax[1].set_title("Filtered")
    plt.show()